In [ ]:
# In this notebook I put the game data into a dataframe in which each row is an indiviual game. I use regex to 
# place the data in understandable columns and then clean and filter the data to get the games I want.

In [1]:
import pandas as pd
import re

In [2]:
# Bringing in the ten csvs created during data gathering, and combining these csvs into a single csv.

cs_1 = pd.read_csv('../data/Caissabase1.csv')
cs_2 = pd.read_csv('../data/Caissabase2.csv')
cs_3 = pd.read_csv('../data/Caissabase3.csv')
cs_4 = pd.read_csv('../data/Caissabase4.csv')
cs_5 = pd.read_csv('../data/Caissabase5.csv')
cs_6 = pd.read_csv('../data/Caissabase6.csv')
cs_7 = pd.read_csv('../data/Caissabase7.csv')
cs_8 = pd.read_csv('../data/Caissabase8.csv')
cs_9 = pd.read_csv('../data/Caissabase9.csv')
cs_10 = pd.read_csv('../data/Caissabase10.csv')

caissabase = pd.concat([cs_1,cs_2,cs_3,cs_4,cs_5,cs_6,cs_7,cs_8,cs_9,cs_10])

In [3]:
# Cleaning up initial dataframe.

caissabase = caissabase.drop(caissabase.columns[0], axis = 1)
caissabase.columns = ['games']

In [4]:
# At this point, all the data for a given game is crammed together in one column. Here, I pull out all the 'event' data and
# put it in an event column. There is likely a way to combine the two regex statements into one, but I left it as is to 
# avoid taking too much time.

caissabase['event'] = caissabase['games'].str.extract('(\[Event[^\]]*\])')
caissabase['event'] = caissabase['event'].str.extract('"([^#]*)"')

In [5]:
# Creating white elo column

caissabase['white_elo'] = caissabase['games'].str.extract('(\[WhiteElo[^\]]*\])')
caissabase['white_elo'] = caissabase['white_elo'].str.extract('"([^#]*)"')

In [6]:
# Creating black elo column

caissabase['black_elo'] = caissabase['games'].str.extract('(\[BlackElo[^\]]*\])')
caissabase['black_elo'] = caissabase['black_elo'].str.extract('"([^#]*)"')

In [7]:
# Dropping games with NaNs for either white or black elo. I need games with known player strengths.

caissabase = caissabase.dropna(axis = 0, how = 'any', subset = ['white_elo','black_elo'])

In [8]:
caissabase = caissabase.astype({'white_elo':'int', 'black_elo':'int'})

In [9]:
# Dropping games with a player below 2000 elo- I want games between players of a high level of skill. I also drop games with
# player elos above 2800- the highest FIDE elo ever in classical chess is 2882, anything much higher than that can be thrown
# out.

caissabase = caissabase[caissabase['white_elo'].between(2000, 2890, 'both')]
caissabase = caissabase[caissabase['black_elo'].between(2000, 2890, 'both')]

In [10]:
# I only want classical games- I don't want data from rapid, blitz, or bullet time controls, which are much faster and allow
# for bad openings to be played without being punished. I want long time control games only, and openings that are played in
# that environment. Unfortunately, there is no easy way to know what time control a game was played in and I had to make do
# by guessing with the event names. I dropped games with event names containing the words 'rapid', 'blitz', and 'bullet' and
# some abbreviations of those as well as any event containing the word 'speed' under the assumption that event was played at
# a faster time control. I also dropped games in the events 'Titled arena' and 'Titled Tuesday', which are online events at
# faster time controls. 
#
# I also only want serious games, so I dropped events containing 'exhibition' and 'simul'. An exhibition game isn't usually
# serious and simul indicates a simultaneous event in which a single player plays many players at the same time- not a
# standard (or likely serious) game. 
# 
# Finally, I only want games in standard 'normal' chess. There are some variants of chess, one of these being Fischer Random
# Chess, also called Chess960, which randomizes the position of the pieces at the start of the game. These games have 
# opening moves not even possible in normal chess and must be dropped. Thus, I dropped any games with events containing '960'
# or 'random'.

caissabase = caissabase[~caissabase['event'].str.contains('rapid| rpd |blitz|bltz|bullet|exhibition| exh |titled arena|titled tuesday|titled tues|speed|simul|960|random', case = False, na = False)]

In [ ]:
# I chose to create columns for elo and event so I could drop a significant number of rows at the start- I was finding it 
# difficult to create other columns without first filtering out a significant number of games because the dataframe was 
# simply so large.
#
# Next I create columns for result, site, date, white player, white fide id, white title, black player, black fide id, black
# title, eco, event date, opening, and variation. I end up not using most of this data in my analysis.

In [11]:
caissabase['result'] = caissabase['games'].str.extract('(\[Result[^\]]*\])')
caissabase['result'] = caissabase['result'].str.extract('"([^#]*)"')

In [12]:
caissabase['site'] = caissabase['games'].str.extract('(\[Site[^\]]*\])')
caissabase['site'] = caissabase['site'].str.extract('"([^#]*)"')

In [13]:
caissabase['date'] = caissabase['games'].str.extract('(\[Date[^\]]*\])')
caissabase['date'] = caissabase['date'].str.extract('"([^#]*)"')

In [14]:
caissabase['white'] = caissabase['games'].str.extract('(\[White[^\]]*\])')
caissabase['white'] = caissabase['white'].str.extract('"([^#]*)"')

In [15]:
caissabase['white_fide_id'] = caissabase['games'].str.extract('(\[WhiteFideId[^\]]*\])')
caissabase['white_fide_id'] = caissabase['white_fide_id'].str.extract('"([^#]*)"')

In [16]:
caissabase['white_title'] = caissabase['games'].str.extract('(\[WhiteTitle[^\]]*\])')
caissabase['white_title'] = caissabase['white_title'].str.extract('"([^#]*)"')

In [17]:
caissabase['black'] = caissabase['games'].str.extract('(\[Black[^\]]*\])')
caissabase['black'] = caissabase['black'].str.extract('"([^#]*)"')

In [18]:
caissabase['black_fide_id'] = caissabase['games'].str.extract('(\[BlackFideId[^\]]*\])')
caissabase['black_fide_id'] = caissabase['black_fide_id'].str.extract('"([^#]*)"')

In [19]:
caissabase['black_title'] = caissabase['games'].str.extract('(\[BlackTitle[^\]]*\])')
caissabase['black_title'] = caissabase['black_title'].str.extract('"([^#]*)"')

In [20]:
caissabase['eco'] = caissabase['games'].str.extract('(\[ECO[^\]]*\])')
caissabase['eco'] = caissabase['eco'].str.extract('"([^#]*)"')

In [21]:
caissabase['event_date'] = caissabase['games'].str.extract('(\[EventDate[^\]]*\])')
caissabase['event_date'] = caissabase['event_date'].str.extract('"([^#]*)"')

In [22]:
caissabase['opening'] = caissabase['games'].str.extract('(\[Opening[^\]]*\])')
caissabase['opening'] = caissabase['opening'].str.extract('"([^#]*)"')

In [23]:
caissabase['variation'] = caissabase['games'].str.extract('(\[Variation[^\]]*\])')
caissabase['variation'] = caissabase['variation'].str.extract('"([^#]*)"')

In [24]:
# Pulling out the move data. This is a bit tricky.

caissabase['moves'] = caissabase['games'].str.extract('(\\n\\n[^%]*)')

In [25]:
caissabase = caissabase.drop(['games'], axis = 1)

In [26]:
# Often the list of moves contains the result of the game at the end. However, the result data is already 
# recorded separately so I can drop this information.
#
# I create a column containing '@' and then concatenated that @ to the end of list of moves. The @ symbol signals the game
# is over. (There's definitely a way to do this without creating a new column, but I couldn't figure it out in a reasonable
# amount of time so this is my unconventional approach). I used @ because chess game notation doesn't use the @ symbol.

caissabase['moves'] = caissabase['moves'].str.replace(r'1-0', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'0-1', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'1/2-1/2', r'')
caissabase['@'] = '@'
caissabase['moves'] = caissabase['moves'] + caissabase['@']

In [27]:
caissabase = caissabase.drop(['@'], axis = 1)

In [28]:
# Here I use a for loop to take the long string of moves in the 'moves' column and separate it out into individual moves.
# The loop starts at 1. It creates an x value '1' and a y value ' 2'. These values will be useful for the loop to know where
# the first move starts in the move sequence and where the second move starts in the move sequence. Let's look at an example
# list of moves in a game:
# 
#      1. e4 c5 2. Nf3 e6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 d6 6. g3 Nc6 7. Bg2 Bd7 8. O-O a6....
# 
# The for loop looks for a '1.'. When it finds it, it begins pulling the string and stops when it reaches either a '.' or a 
# '@'. I have it stop at the next '.' because chess game notation only uses '.' after a move number, like '2.' or '45.' so I
# always know a '.' signals the next move. I also tell it to stop pulling data if it finds a '@'- remember, I concatenated
# '@' symbols at the end of every game notation, so if it sees an '@' the game is over, so it stops pulling the string  
# (although looking back on this, I'm not ever sure I need to the @ thing at all- that's what I get for trying to be clever).
# In the example above, regex pulls the following string:
#
# '1. e4 c5 2'
#
# This is mostly what I want, but I really care about the moves and not the move numbers- at this point I know this is data
# on move 1. So, I replace the y value of ' 2' with '' and the x value of '1' and a '.' with ''. After  all that, this is 
# what I get:
# 
# ' e4 c5'.
# 
# This is what I want- just the moves on move 1! The for loop creates a column named x (in this case, '1') and moves on. The
# for loop will continue to x = 2 and do the same with move 2, and so on. I stop on move 20 because I'm trying to analyze 
# the opening and I don't think going deeper than move 20 is nesseccary.

for x in range (1,21):
    y = ' ' + str(x + 1)
    x = str(x)
    caissabase[x] = caissabase['moves'].str.extract('('+x+'\.[^@.]*)')
    caissabase[x] = caissabase[x].str.replace(y, '')
    caissabase[x] = caissabase[x].str.replace(x + '\.', '')

<ipython-input-28-336d59ff31a9>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  caissabase[x] = caissabase[x].str.replace(x + '\.', '')


In [29]:
caissabase = caissabase.drop(['moves'], axis = 1)

In [30]:
# I have the move data seperated by move, but I also want to seperate each move into white's move and black's move. Let's
# use the example from our previous step:
#
# ' e4 c5'
#
# This for loop strips the string to get 'e4 c5' and splits the string based on the space, to get 'e4' and 'c5'. These two
# moves are then saved to seperate columns '1w' and '1b' (for move one white and move one black).

for x in range (1,21):
    x = str(x)
    w = x + 'w'
    b = x + 'b'
    caissabase[x] = caissabase[x].str.strip()
    caissabase[[w, b]] = caissabase[x].str.split(' ', n = 1, expand = True)

In [31]:
# With two half moves columns created from each move column, we don't need the move columns anymore.

caissabase = caissabase.drop(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'], axis = 1)

In [32]:
caissabase['date'] = pd.to_datetime(caissabase['date'], errors = 'coerce')

In [33]:
# As time has gone on, opening theory has progressed and players have generally become stronger. Because of this, we are
# throwing out any games before the year 2000.

caissabase = caissabase.loc[caissabase['date']>='2000-01-01']

In [34]:
# I drop any games with NaNs within the first 4 moves. If a game ends, any move columns after the game ends will be NaN.
# This drops games that ended really quickly and can skew the results. For example, I don't want the game 1.e4 1-0 to be in
# my database- there are a variety of reasons a game could end without black even moving, perhaps for example the black 
# player forfeited. Whatever the case, this isn't a 'real' win for the opening 1.e4 and I don't want it in my data.

caissabase = caissabase.dropna(axis = 0, how = 'any', subset = ['1w','1b','2w','2b','3w','3b','4w','4b'])

In [35]:
# There were some first moves by white and black that are actually impossible to be played- these are dropped.

caissabase = caissabase[caissabase['1w'].isin(['Bh3','Nd6+','f5','Nf5']) == False]
caissabase = caissabase[caissabase['1b'].isin(['{ Note this is not a standard game of chess as castling was not\nallowed } 1','f62']) == False]

In [37]:
# Some games have a '*' for their result- I think this means the game never finished. These games are dropped.

caissabase = caissabase.loc[caissabase['result'] != '*']

In [38]:
caissabase = caissabase.reset_index(drop =True)

In [39]:
# I save the cleaned and filtered database back to a csv.

caissabase.to_csv('../data/caissabase_df.csv')